<h3><span style="font-weight:bold"> 검색어 뉴스 정보(제목, 게시일자, 신문사) 가져오기 </span></h3>

<h4><span style="font-weight:bold"> 1page 정보 가져오기 </span></h4>

In [ ]:
# 모듈 불러오기
from datetime import datetime
from datetime import timedelta
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

# SSL 경고창 없애기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

# 정보 입력하기
search_key = "포스코홀딩스"
now_date = datetime.now().strftime("%y%m%d")
i = 0
p = {
    "where": "news",
    "query": search_key,
    "start": i * 10 + 1
}
url = "https://search.naver.com/search.naver?"

# 1페이지 접속하기
res = requests.get(url, params=p, verify=False)
res.text

# 파싱하기
# 뉴스 제목 가져오기
soup = bs(res.text, "html.parser")
titles = soup.select(".news_tit")

# 신문사 정보 가져오기
press_group = soup.select("a.info.press")

# 게시 일자 가져오기
time_info = soup.select("span.info")

i = 0
results = []
for i in range(10):
    title = titles[i].text
    date = time_info[i].text
    press = press_group[i].text
    info = {"제목":title, "게시일자":date, "신문사":press}
    results.append(info)

# 엑셀로 전송하기
pd.DataFrame(results).to_excel("outputs/files/{}_{}검색 뉴스 타이틀 정보.xlsx".format(now_date, search_key))

<h4><span style="font-weight:bold"> 1~10page 정보 가져오기 </span></h4>

In [1]:
# 모듈 불러오기
from datetime import datetime
from datetime import timedelta
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

# SSL 경고창 없애기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

# 정보 입력하기
search_key = "포스코홀딩스"
now_date = datetime.now().strftime("%y%m%d")
results = []

for i in range(10):
    print("{}페이지 크롤링 중....".format(i), end="\r")
    p = {
        "where": "news",
        "query": search_key,
        "start": i * 10 + 1
    }
    url = "https://search.naver.com/search.naver?"

    # 페이지 접속하기
    res = requests.get(url, params=p, verify=False)

    # 파싱하기
    # 뉴스 제목 가져오기
    soup = bs(res.text, "html.parser")
    titles = soup.select(".news_tit")

    # 신문사 정보 가져오기
    press_group = soup.select("a.info.press")

    # 게시 일자 가져오기
    time_info = soup.select("span.info")
    for idx, ti in enumerate(time_info):
        if ti.text.find("면") != -1:
            del time_info[idx]

    for j in range(10):
        title = titles[j].text
        date = time_info[j].text
        if date.find("분 전") != -1:
            now = datetime.now()
            gap = int(date.split("분 전")[0])
            press_date = now - timedelta(minutes=gap)
            date = press_date.strftime("%Y.%m.%d")
        elif date.find("시간 전") != -1:
            now = datetime.now()
            gap = int(date.split("시간 전")[0])
            press_date = now - timedelta(hours=gap)
            date = press_date.strftime("%Y.%m.%d")
        elif date.find("일 전") != -1:
            now = datetime.now()
            gap = int(date.split("일 전")[0])
            press_date = now - timedelta(days=gap)
            date = press_date.strftime("%Y.%m.%d")
        else:
            date = date[:-1]
        press = press_group[j].text
        info = {"제목":title, "게시일자":date, "신문사":press}
        results.append(info)

file_name = "./outputs/files/{}_{}검색 뉴스 타이틀 정보.xlsx".format(now_date, search_key)

# 엑셀로 전송하기
df = pd.DataFrame(results)
df.to_excel(file_name, index=False)

# 결과 확인하기
display(df)
print("{}_{}_뉴스 검색 크롤링 완료!".format(now_date, search_key))
print("저장된 파일명 : {}".format(file_name))

,제목,게시일자,신문사
0,"""포스코는 국민기업이다""…포항 시민단체 책 발간",2023.01.16,연합뉴스
1,"포스코그룹, 2030년까지 유니콘 15개 육성한다",2023.01.12,한국경제언론사 선정
2,"포스코, 실물경제 불황에도 실적개선 등 시장평가 '긍정적'",2023.01.16,뉴데일리
3,"포스코, 美 실리콘밸리에 스타트업 창업 육성센터 가동",2023.01.12,뉴시스
4,"[stock&톡]에너지 업고 증시 나서는 포스코인터…성장 기대감, 박스권도 뚫어...",2023.01.16,뉴스웨이
...,...,...,...
95,"코스피 끌어 올린 외인, ‘삼성전자’ 사고 ‘포스코케미칼’ 팔고",2023.01.16,이투데이
96,"포스코그룹, 사장단 인사 단행",2022.12.28,광주매일신문
97,"포스코그룹 사장단 인사···정탁, 부회장 승진해 통합 인터내셔널 신임 대표...",2022.12.27,경향신문
98,"포스코그룹, 사장단 인사 단행… 정탁 부회장 승진",2022.12.27,뉴데일리


230117_포스코홀딩스_뉴스 검색 크롤링 완료!
저장된 파일명 : ./outputs/files/230117_포스코홀딩스검색 뉴스 타이틀 정보.xlsx


In [ ]:
# 모듈 불러오기
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

search_str = "포스코홀딩스"
# URL Decoder/Encoder 사이트 = https://meyerweb.com/eric/tools/dencoder/

result = []
start_no = -9
# base_url = "https://search.naver.com/search.naver?where=news&query="+search_str+"&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=4&ds=&de=&docid=&nso=so%3Ar%2Cp%3A1d%2Ca%3Aall&mynews=0&refresh_start=0&related=0&start="
# base_url = "https://search.naver.com/search.naver?where=news&query=포스코&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=4&ds=&de=&docid=&nso=so%3Ar%2Cp%3A1d%2Ca%3Aall&mynews=0&refresh_start=0&related=0&start="
base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query="+search_str+"&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=56&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start="
now_date = datetime.datetime.now().strftime("%Y-%m-%d")
ssl_verify = False
# SSL 경고창 없애기
requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

for i in range(1, 11):
    print("네이버 뉴스 {}번째 페이지 크롤링중... (검색어 : {})".format(i, search_str), end="\r")
    start_no = start_no + 10
    url = base_url + str(start_no)
    html = requests.get(url, verify = False)
    soup = bs(html.text, "html.parser")
    news_arr = soup.select(".news_tit")
    for news_title in news_arr:
        this_result = {
            "크롤링 날짜" : now_date,
            "제목" : news_title.text,
            "링크" : news_title.get("href")
        }
        result.append(this_result)
df = pd.DataFrame(result)
# inplace = True는 변경한 df를 기존 df에 저장한다는 의미
# df.drop_duplicates("제목", inplace=True)
# df.reset_index(inplace=True)
# df.drop(["index"], axis=1, inplace=True)
file_name = "./outputs/"+str(now_date)+"_네이버 뉴스 크롤링 결과_검색어_"+search_str+".xlsx"
# 엑셀로 저장시 df의 인덱스를 없애고 싶으면 index=False, header/startrow/startcol 은 없어도 됨
df.to_excel(
    file_name,
    # header = True,
    index = False,
    # startrow = 0, 
    # startcol = 0
    )
display(df)
print("네이버 뉴스 크롤링 및 저장 완료")
print("저장된 파일명 : "+file_name)